In [ ]:
import pandas as pd
import numpy as np
import itertools
import time
import openai
import os, sys
from tqdm.notebook import tqdm

##Import openai key
openai.api_key = '###########################'

In [ ]:
df_sally = pd.read_csv('Data/Sally-Ann_prompt.csv', index_col = 0)
df_crayon = pd.read_csv('Data/Smarties_prompt.csv', index_col = 0)

## Define openai function

In [1]:
def get_response(prompt, max_tokens):
    rgpt4 = openai.ChatCompletion.create(
      messages=[{"role": "user", "content": prompt}],
      model = 'gpt-4-0613',
      temperature=0,
      max_tokens=max_tokens,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0)
    rturbo = openai.ChatCompletion.create(
      messages=[{"role": "user", "content": prompt}],
      model = 'gpt-3.5-turbo-0301',
      temperature=0,
      max_tokens=max_tokens,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0)
    rdvc = openai.Completion.create(
      prompt = prompt,
      model = 'text-davinci-003',
      temperature=0,
      max_tokens=max_tokens,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0	)
    return rgpt4['choices'][0]['message']['content'], rturbo['choices'][0]['message']['content'], rdvc['choices'][0]['text']

In [ ]:
def run(col_prompt, col_response, max_token, df):
    col_response_gpt4 = col_response + '_gpt4_pred'
    col_response_turbo = col_response + '_turbo_pred'
    col_response_davinci = col_response + '_davinci_pred'
    df[col_response_gpt4] = ''
    df[col_response_turbo] = ''
    df[col_response_davinci] = ''
    max_attempts = 10
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        if pd.isna(row[col_response]) or row[col_response] == '':
            for attempt in range(max_attempts):
                try:
                    response = get_response(row[col_prompt], max_token)
                    df.at[index, col_response_gpt4] = response[0]
                    df.at[index, col_response_turbo] = response[1]
                    df.at[index, col_response_davinci] = response[2]
                    break  # if the attempt was successful, break out of the retry loop
                except Exception as e:
                    print(f"Encountered an error at index {index}. Retrying attempt {attempt+1}...")
                    time.sleep(5)  # pause execution for 5 seconds before next attempt, adjust as needed
            else:  # this else clause executes after the for loop finishes without a break (i.e., all attempts failed)
                print(f"Failed to get a response after {max_attempts} attempts at index {index}. Moving on to next row.")
    return df


## Generate answers:
**prompt column names:**

Question and Answer = qa_prompt,

Completion = comp_prompt,

Multiple choice = mc_prompt,

Fill in Blank = fb_prompt,

True or False = tf_prompt,

True or False with reason = tfr_prompt<br>
**max tokens**

qa: 50, comp: 50, mc: 2, fb: 10, tf: 20, tfr: 100

QA

In [ ]:
df_sally = run('qa_prompt', 'qa', 50, df_sally)

In [ ]:
df_crayon = run('qa_prompt', 'qa', 50, df_crayon)

  0%|          | 0/180 [00:00<?, ?it/s]

Completion

In [ ]:
df_sally = run('comp_prompt', 'comp', 50, df_sally)

  0%|          | 0/180 [00:00<?, ?it/s]

Encountered an error at index 0. Retrying attempt 1...


In [ ]:
df_crayon = run('comp_prompt', 'comp', 50, df_crayon)

  0%|          | 0/180 [00:00<?, ?it/s]

Multiple choice

In [ ]:
df_sally = run('mc_prompt', 'mc', 2, df_sally)

  0%|          | 0/180 [00:00<?, ?it/s]

In [ ]:
df_crayon = run('mc_prompt', 'mc', 2, df_crayon)

  0%|          | 0/180 [00:00<?, ?it/s]

Fill in Blank

In [ ]:
df_sally = run('fb_prompt', 'fb', 10, df_sally)

  0%|          | 0/180 [00:00<?, ?it/s]

In [ ]:
df_crayon = run('fb_prompt', 'fb', 10, df_crayon)

  0%|          | 0/180 [00:00<?, ?it/s]

True or False

In [ ]:
df_sally = run('tf_prompt', 'tf', 20, df_sally)

  0%|          | 0/180 [00:00<?, ?it/s]

In [ ]:
df_crayon = run('tf_prompt', 'tf', 20, df_crayon)

  0%|          | 0/180 [00:00<?, ?it/s]

True or False with Reason

In [ ]:
df_sally = run('tfr_prompt', 'tfr', 100, df_sally)

  0%|          | 0/180 [00:00<?, ?it/s]

Encountered an error at index 175. Retrying attempt 1...


In [ ]:
df_crayon = run('tfr_prompt', 'tfr', 100, df_crayon)

  0%|          | 0/180 [00:00<?, ?it/s]

Encountered an error at index 55. Retrying attempt 1...
